In [1]:
import numpy as np, cv2

In [2]:
def draw_histo(hist, size=(200, 256)):
    hist_img = np.full((size[0], size[1]), 255, dtype=np.uint8)
    hist = cv2.normalize(hist, None, 0, size[0], cv2.NORM_MINMAX).flatten()

    bin_width = int(np.round(size[1] / hist.shape[0]))
    for i in range(hist.shape[0]):
        x = i * bin_width
        y = size[0] - int(hist[i])
        cv2.rectangle(hist_img, (x, y), (x + bin_width - 1, size[0]), 0, -1)
    return hist_img
def search_value_idx(hist, bias = 0):
    for i in range(hist.shape[0]):
        idx = np.abs(bias - i)                     # 검색 위치 (처음 또는 마지막)
        if hist[idx] > 0:  return idx                             # 위치 반환
    return -1                                      # 대상 없으면 반환

image = cv2.imread("images/hist_stretch.jpg", cv2.IMREAD_GRAYSCALE)   # 영상읽기
if image is None: raise Exception("영상 파일 읽기 오류")

bsize, ranges = [64], [0,256]                        # 계급 개수 및 화소 범위
hist = cv2.calcHist([image], [0], None, bsize, ranges)

bin_width  = ranges[1]/bsize[0]                      # 계급 너비
high = search_value_idx(hist, bsize[0] - 1) * bin_width
low  = search_value_idx(hist, 0) * bin_width

idx = np.arange(0, 256)
idx = (idx - low) * 255/(high-low)	# 수식 적용하여 인덱스 생성
idx[0:int(low)] = 0
idx[int(high+1):] = 255

dst = cv2.LUT(image, idx.astype('uint8'))
# dst = idx.astype('uint8')[image]                #

## 룩업 테이블 사용하지 않고 직접 구현
# dst = np.zeros(image.shape, dtype=image.dtype)
# for i in range(dst.shape[0]):
#     for j in range(dst.shape[1]):
#         dst[i,j] = idx[image[i,j]]

hist_dst = cv2.calcHist([dst], [0], None, bsize, ranges)  # 결과 영상 히스토그램 재계산
hist_img = draw_histo(hist, (200,360))          # 원본 영상 히스토그램 그리기
hist_dst_img = draw_histo(hist_dst,(200,360))  # 결과 영상 히스토그램 그리기

print("high_value = ", high)
print("low_value = " , low)
cv2.imshow("image", image);         cv2.imshow("hist_img", hist_img)
cv2.imshow("dst", dst);             cv2.imshow("hist_dst_img", hist_dst_img)
cv2.waitKey(0)

high_value =  180.0
low_value =  52.0


-1